# 02 - Baseline ModelThis notebook loads the baseline diffusion model (FP32) and establishes performance benchmarks.

In [ ]:
import osimport torchimport timefrom diffusers import StableDiffusionPipelineimport numpy as npimport jsonimport config as cfgimport psutil

## Load Baseline Model (FP32)

In [ ]:
print(f"Loading baseline model: {cfg.MODEL_NAME}")device = "cuda" if torch.cuda.is_available() else "cpu"print(f"Using device: {device}")pipe = StableDiffusionPipeline.from_pretrained(cfg.MODEL_NAME, torch_dtype=torch.float32)pipe = pipe.to(device)print("Model loaded successfully!")

## Generate Baseline Images

In [ ]:
with open(os.path.join(cfg.OUTPUT_DIR, 'test_prompts.json'), 'r') as f:    test_prompts = json.load(f)baseline_results = []for i, prompt in enumerate(test_prompts[:5]):    start_time = time.time()    image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, height=512, width=512).images[0]    end_time = time.time()    latency = end_time - start_time    baseline_results.append({'prompt': prompt, 'latency': latency})    image.save(os.path.join(cfg.IMAGES_DIR, f'baseline_{i}.png'))

In [ ]:
latencies = [r['latency'] for r in baseline_results]avg_latency = np.mean(latencies)print(f"Baseline Average Latency: {avg_latency:.2f} seconds")with open(os.path.join(cfg.RESULTS_DIR, 'baseline_results.json'), 'w') as f:    json.dump({'avg_latency': float(avg_latency), 'results': baseline_results}, f)